# Scrape Seals Over Calligraphy

Operates over the calligraphy section of cidianwang, hopefully

In [2]:
import csv
import os
import numpy as np
import requests
from bs4 import BeautifulSoup, SoupStrainer
import re
import cssutils
from PIL import Image
import urllib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import io
from wand.api import library
import wand.color
import wand.image
import time
import sys
import httplib2
from scipy.ndimage import zoom
import logging

# logging.basicConfig(stream=sys.stdout)
formatter = logging.Formatter("[%(asctime)s] [%(levelname)s] %(message)s")
handler = logging.FileHandler("log_3.log")
handler.setFormatter(formatter)
handler2 = logging.StreamHandler(sys.stdout)
handler2.setFormatter(formatter)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.addHandler(handler)
logger.addHandler(handler2)
# logging.debug("test")

data_dir = './source'
# scrape_url_base = 'https://hanziyuan.net/#'

scrape_url_base = 'https://www.cidianwang.com/shufa/'
new_image_filetype = 'png'
retry = True

manual = False

logger.info(f"Logging activated [retry {retry}] [manual {manual}] [target {scrape_url_base}]")
# if not os.path.exists(os.path.join(data_dir, log)):
#     log = open(os.path.join(data_dir, log), 'x', encoding='utf8')

[2023-11-06 20:58:09,053] [INFO] Logging activated [retry True] [manual False] [target https://www.cidianwang.com/shufa/]


In [3]:
def load_data(csv):
    data = np.genfromtxt(os.path.join(data_dir, csv), delimiter=',', encoding='utf8', dtype=None)
    logger.info(f"Imported {data.shape[0]} characters from index {csv}")
    
    return data

data = load_data("hsk.csv")

[2023-11-06 20:58:11,542] [INFO] Imported 1076 characters from index hsk.csv


In [11]:
headers = {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
        'Referer': 'https://www.cidianwang.com/'
    }
    
try:
    url = f"https://search.cidianwang.com/?m=8&q=文&z=输入书法家&y=4"
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    print(soup.prettify())
except:
    e = sys.exc_info()[0]
    logger.error(f'Char 文. Failed to obtain with error: {e}')

[2023-11-06 21:20:28,660] [DEBUG] Starting new HTTPS connection (1): search.cidianwang.com:443
[2023-11-06 21:20:29,841] [DEBUG] https://search.cidianwang.com:443 "GET /?m=8&q=%E6%96%87&z=%E8%BE%93%E5%85%A5%E4%B9%A6%E6%B3%95%E5%AE%B6&y=4 HTTP/1.1" 302 2866
[2023-11-06 21:20:29,843] [DEBUG] Starting new HTTPS connection (1): www.cidianwang.com:443
[2023-11-06 21:20:31,181] [DEBUG] https://www.cidianwang.com:443 "GET /shufa/wen6677_zs.htm HTTP/1.1" 200 None
<!DOCTYPE html>
<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   文字篆书写法_文篆书怎么写好看_文书法图片_词典网
  </title>
  <meta content="文，文书法，文怎么写，文书法图片，文毛笔字，文行书，文草书，文楷书，文隶书，文篆书，文书法字典" name="keywords"/>
  <meta content="文怎么写好看，文毛笔怎么写，文书法图片，文名家书法，文篆书书法欣赏，文篆书书法字帖，文的行书写法，文的草书写法，文的楷书写法，文的隶书写法。" name="description">
   <meta content="format=html5; url=https://m.cidianwang.com/shufa/wen6677_zs.htm" http-equiv="mobile-agent"/>
   <link href="/css/style.css" rel="stylesheet" type="text/css"/>
   <script src="/c

In [19]:
results = soup.findAll("img", {"class": "lazyload"})
urls = [result.attrs["data-src"] for result in results]
# print(urls)

headers2 = {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
        'Referer': 'https://www.cidianwang.com/',
        "Host": "c.cidianwang.com"
    }

for i, url in enumerate(urls):
    if not re.findall("zhuanshu", url):
        continue
    print(i, url)
    img = Image.open(requests.get(url, stream=True, headers=headers2).raw)
    img.save(f"wen-{i}.png")

0 https://c.cidianwang.com/file/shufa/zhuanshu/ligangtian/201972592947a7b67.jpg
[2023-11-06 21:39:58,797] [DEBUG] Starting new HTTPS connection (1): c.cidianwang.com:443
[2023-11-06 21:40:00,064] [DEBUG] https://c.cidianwang.com:443 "GET /file/shufa/zhuanshu/ligangtian/201972592947a7b67.jpg HTTP/1.1" 200 12326
1 https://c.cidianwang.com/file/shufa/zhuanshu/laichusheng/2019725926355310.jpg
[2023-11-06 21:40:00,074] [DEBUG] Starting new HTTPS connection (1): c.cidianwang.com:443
[2023-11-06 21:40:01,109] [DEBUG] https://c.cidianwang.com:443 "GET /file/shufa/zhuanshu/laichusheng/2019725926355310.jpg HTTP/1.1" 200 10180
2 https://c.cidianwang.com/file/shufa/zhuanshu/2018841833845f5f.jpg
[2023-11-06 21:40:01,116] [DEBUG] Starting new HTTPS connection (1): c.cidianwang.com:443
[2023-11-06 21:40:02,222] [DEBUG] https://c.cidianwang.com:443 "GET /file/shufa/zhuanshu/2018841833845f5f.jpg HTTP/1.1" 200 22193
3 https://c.cidianwang.com/file/shufa/zhuanshu/2018841829473b63f.jpg
[2023-11-06 21:40:0

In [ ]:
for i, url in enumerate(urls):
    if not re.findall("zhuanshu", url):
        continue
    img = Image.open(f"./testims/wen-{i}".png)
    

In [9]:
# https://search.cidianwang.com/?m=8&q=文&z=输入书法家&y=4

def searchShufa(character):
    """character: any han character"""
    headers = {
        'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36",
        'Referer': 'https://www.cidianwang.com/'
    }
    
    try:
        url = f"https://search.cidianwang.com/?m=8&q={character}&z=输入书法家&y=4"
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.content, "html.parser")
        print(soup)
    except:
        e = sys.exc_info()[0]
        logger.error(f'Char {character}. Failed to obtain with error: {e}')


searchShufa("文")

[2023-11-06 21:15:03,812] [DEBUG] Starting new HTTPS connection (1): search.cidianwang.com:443
[2023-11-06 21:15:04,919] [DEBUG] https://search.cidianwang.com:443 "GET /?m=8&q=%E6%96%87&z=%E8%BE%93%E5%85%A5%E4%B9%A6%E6%B3%95%E5%AE%B6&y=4 HTTP/1.1" 302 2866
[2023-11-06 21:15:04,921] [DEBUG] Starting new HTTPS connection (1): www.cidianwang.com:443
[2023-11-06 21:15:06,270] [DEBUG] https://www.cidianwang.com:443 "GET /shufa/wen6677_zs.htm HTTP/1.1" 200 None
<!DOCTYPE html>
<html><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><title>文字篆书写法_文篆书怎么写好看_文书法图片_词典网</title><meta content="文，文书法，文怎么写，文书法图片，文毛笔字，文行书，文草书，文楷书，文隶书，文篆书，文书法字典" name="keywords"/><meta content="文怎么写好看，文毛笔怎么写，文书法图片，文名家书法，文篆书书法欣赏，文篆书书法字帖，文的行书写法，文的草书写法，文的楷书写法，文的隶书写法。" name="description"><meta content="format=html5; url=https://m.cidianwang.com/shufa/wen6677_zs.htm" http-equiv="mobile-agent"/><link href="/css/style.css" rel="stylesheet" type="text/css"/><script src="/css/newcommon.js" type="text/javas